In [10]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import *
from tensorflow.keras.backend import clear_session

import numpy as np
import matplotlib.pyplot as plt
from random import randint
from ..transformer import TransformerBlock, TokenAndPositionEmbedding

ImportError: attempted relative import with no known parent package

In [2]:
from utils import load_dataset

X_train, y_train, X_test, y_test = load_dataset('FOX')

/tmp/FOX_train.h5


In [6]:
transposed = np.array([x.T for x in X_train])

In [1]:
sample_size = 10000

embed_dim = 4
num_heads = 2 # Number of attention heads
ff_dim = 64 # Hidden layer size in feed forward network inside transformer

clear_session()

inputs = Input(shape=(1000, 4))
# inputs = Input(shape=(1000,))

embedding_layer = TokenAndPositionEmbedding(1000, sample_size, 4)
x = embedding_layer(inputs)
# x = inputs

transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = GlobalAveragePooling1D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.1)(x)
outputs = Dense(31, activation='relu')(x)

model = Model(inputs, outputs)

# model.summary()

NameError: name 'clear_session' is not defined

In [6]:
%%time
model.compile(optimizer=Adam(), loss='mae')
history = model.fit(
    transposed[:10000], y_train[:10000], epochs=10
)

Epoch 1/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0005
Epoch 2/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0002
Epoch 3/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0002
Epoch 4/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0001
Epoch 5/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0002
Epoch 6/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0001
Epoch 7/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0002
Epoch 8/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0000
Epoch 9/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0002
Epoch 10/10
313/313 [==============================] - 12s 37ms/step - loss: 158.0001


In [ ]:
start = 0
end = 10

plt.title('all epochs')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(range(start, end), history.history['loss'][start:end])
plt.show()

In [8]:
model.evaluate(np.array([x.T for x in X_test]), y_test)

313/313 [==============================] - 5s 15ms/step - loss: 173.6339


173.63389587402344

In [1]:
from tensorflow.keras.models import load_model
from evaluate import AUC
model = load_model('model_saves/transformer.h5')

In [7]:
%%time
print(AUC(model, transposed[:10000], y_train[:10000]))

{'false negative': 5717, 'true negative': 4283}
CPU times: user 3 s, sys: 257 ms, total: 3.26 s
Wall time: 6.07 s


In [ ]:
n = randint(0, len(y_test))

pred = model.predict(np.expand_dims(X_test[n].T, 0))

print(n, '\n', pred, '\n', y_test[n])

model.save('model_saves/transformer-model')